In [1]:
import ChordDetection
import os
import pandas as pd
import numpy as np

/home/yvson_eng/Documents/python/chord_detection/venv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/yvson_eng/Documents/python/chord_detection/venv/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If y

In [2]:
chord_list = ['A','A#','B','C','C#','D','D#','E','F','F#','G','G#','Am','A#m','Bm','Cm','C#m','Dm','D#m','Em','Fm','F#m','Gm','G#m','N']

In [3]:
song_path = "./music/Blue Ain't Your Color.mp3"

In [4]:
DC = ChordDetection.DetectChords()
DC.build_cnn_extractor()
DC.build_crf()

if os.path.exists(song_path.replace(".mp3", ".wav")):
    os.remove(song_path.replace(".mp3", ".wav"))

DC.mp3_to_wav(song_path)

2023-04-24 08:29:46.966085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 08:29:47.030636: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 08:29:47.030687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 08:29:47.031672: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate


Mp3 to wav conversion.


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab


Saving wav file to data...



Error while decoding stream #0:0: Invalid data found when processing input
size=   42593kB time=00:04:07.22 bitrate=1411.4kbits/s speed=1.01e+03x    
video:0kB audio:42592kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000179%


In [5]:
predictions_shape, final_preds = DC.predict_seq()



Making predictions - this might take 1-3 minutes. Please wait...



2023-04-24 08:29:53.842171: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204
2023-04-24 08:30:36.146530: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Predictions are created.


In [6]:
start = 0
end = predictions_shape[1]
time_list = np.round(np.arange(start//10, end//10, 0.1), 1)

In [7]:
final_preds_chords = [chord_list[int(chord_idx)] for chord_idx in final_preds]
final_preds_chords = final_preds_chords[0: len(time_list)]

In [8]:
def generate_chord_detection_csv_files(time_list, final_preds_chords):
    start = time_list[0]
    df_list = []
    for i in range(0, len(time_list)):
        if(i == len(time_list) - 2): #penultimo seg
            df_list.append([start, time_list[i + 1], final_preds_chords[i]])
            break
        elif(final_preds_chords[i] != final_preds_chords[i + 1]):
                df_list.append([start, time_list[i], final_preds_chords[i]])
                start = time_list[i + 1]

    for i in range(1, len(df_list)):
        df_list[i][0] = df_list[i-1][1]

    df_list = [tuple(seg) for seg in df_list]

    df = pd.DataFrame(df_list, columns=["start", "end", "chord"])

    return df

In [9]:
generate_chord_detection_csv_files(time_list, final_preds_chords)

,start,end,chord
0,0.0,4.1,N
1,4.1,8.8,G
2,8.8,13.0,Am
3,13.0,15.3,C
4,15.3,17.2,D
...,...,...,...
83,217.4,218.2,Am
84,218.2,222.1,G
85,222.1,222.6,Am
86,222.6,228.1,G
